In [37]:
from google.colab import drive

In [38]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
train_data = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/rusentitweet_train.csv')
test_data = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/rusentitweet_test.csv')

true_labels = ['positive', 'negative']
filtered_train_data = train_data[train_data['label'].isin(true_labels)]
filtered_test_data = test_data[test_data['label'].isin(true_labels)]

filtered_train_data.head(6)

In [ ]:
import re
# Очистка текста
def cleaner(tweet):
  #Удаление ссылок на аккаунты
  tweet = re.sub(r'@[\w]+', '', tweet)

  #Удаление обычных ссылок
  tweet = re.sub(r'https?://\S+', '', tweet)

  #Удаление спецсимволов "^", "&", "*", "|", "/"
  tweet = re.sub(r'[\^&*/|]', '', tweet)

  #Удаление хэштегов
  tweet = re.sub(r'#\w+', '', tweet)

  return tweet

cleaned_train_data = filtered_train_data.copy()
cleaned_test_data = filtered_test_data.copy()

cleaned_train_data['text'] = cleaned_train_data['text'].apply(cleaner)
cleaned_test_data['text'] = cleaned_test_data['text'].apply(cleaner)

cleaned_train_data.head(6)

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
#Осуществим стемминг подготовленного набора данных
stemmer = SnowballStemmer('russian')
def stemming(tweet):
  words_arr = tweet.split()
  words_arr = list(map(lambda w: stemmer.stem(w), words_arr))
  return ' '.join(words_arr)

# Применение стемминга
stemmed_train_data = cleaned_train_data.copy()
stemmed_test_data = cleaned_test_data.copy()
stemmed_train_data['text'] = stemmed_train_data['text'].apply(stemming)
stemmed_test_data['text'] = stemmed_test_data['text'].apply(stemming)

stemmed_train_data.head(6)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#Преобразование в мешок слов
vectorizer = CountVectorizer()
stemmed_train_data_counts = vectorizer.fit_transform(stemmed_train_data['text'])
stemmed_test_data_counts = vectorizer.transform(stemmed_test_data['text'])
print(stemmed_train_data_counts)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
#TF-IDF
tfidf = TfidfTransformer()
#Обучение на тренировочных данных
d_train_tfidf = tfidf.fit_transform(stemmed_train_data_counts)
#Преобразование текста в Tf-Idf матрицу на тестовых данных
d_test_tfidf = tfidf.transform(stemmed_test_data_counts)

print(d_test_tfidf)

In [ ]:
#Обучаем модели логистической регрессии и случайного леса на обучающей выборке, применяем их к тестовым данным
label_maps = {"positive":1, "negative":0}

#Сопоставление меток с числами
D_train_encoded = stemmed_train_data['label'].map(label_maps)
D_test_encoded = stemmed_test_data['label'].map(label_maps)

from sklearn.linear_model import LogisticRegression
#Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(d_train_tfidf, D_train_encoded)

from sklearn.ensemble import RandomForestClassifier
#Случайный лес
rand_forest = RandomForestClassifier()
rand_forest.fit(d_train_tfidf, D_train_encoded)

#Предсказания
predict_lin_reg_train = log_reg.predict(d_train_tfidf)
predict_rand_for_train = rand_forest.predict(d_train_tfidf)

predict_lin_reg_test = log_reg.predict(d_test_tfidf)
predict_rand_for_test = rand_forest.predict(d_test_tfidf)

from sklearn.metrics import classification_report
#выводим полученные результаты по показателям
print("Обучающие данные\n")
print("Линейная регрессия\n", classification_report(D_train_encoded,predict_lin_reg_train))
print("Случайный лес\n", classification_report(D_train_encoded,predict_rand_for_train))

print("Тестовые данные\n")
print("Линейная регрессия\n", classification_report(D_test_encoded,predict_lin_reg_test))
print("Случайный лес\n", classification_report(D_test_encoded,predict_rand_for_test))

In [52]:
#Выявляем наиболее важные признаки(слова)
n = 15

log_reg_coef = log_reg.coef_[0]
log_reg_ind = abs(log_reg_coef).argsort()[::-1]
log_reg_names = vectorizer.get_feature_names_out()
log_reg_top_names = [log_reg_names[index] for index in log_reg_ind[:n]]
print("Топ 25 наиболее значимых слов для логистической регрессии:\n", log_reg_top_names)

rand_forest_coef = rand_forest.feature_importances_
rand_forest_ind = abs(rand_forest_coef).argsort()[::-1]
rand_forest_names = vectorizer.get_feature_names_out()
rand_forest_top_names = [rand_forest_names[index] for index in rand_forest_ind[:n]]
print("Топ 25 наиболее значимых слов для случайного леса:\n", rand_forest_top_names)


Топ 25 наиболее значимых слов для логистической регрессии:
 ['не', 'любл', 'хорош', 'пиздец', 'блят', 'красив', 'нет', 'нрав', 'мил', 'классн', 'нах', 'рад', 'крут', 'хорошо', 'прекрасн']
Топ 25 наиболее значимых слов для случайного леса:
 ['не', 'так', 'любл', 'хорош', 'эт', 'что', 'как', 'пиздец', 'нет', 'красив', 'ты', 'блят', 'мо', 'все', 'на']


In [ ]:
from pymystem3 import Mystem
#Лемматизация
m = Mystem()
def lemmatization(tweet):
  lem = m.lemmatize(tweet)
  lemmas = []
  for l in lem:
    if l.isalnum():
      lemmas.append(l)
  lem_tweet = ' '.join(lemmas)
  return lem_tweet

# Применение лемматизации
lem_train_data = cleaned_train_data.copy()
lem_test_data = cleaned_test_data.copy()
lem_train_data['text'] = lem_train_data['text'].apply(lemmatization)
lem_test_data['text'] = lem_test_data['text'].apply(lemmatization)

lem_train_data.head(50)

In [ ]:
#Всё то же самое, но с лемматизацией

#Преобразование в мешок слов
vectorizer = CountVectorizer()
lem_train_data_counts = vectorizer.fit_transform(lem_train_data['text'])
lem_test_data_counts = vectorizer.transform(lem_test_data['text'])
print(lem_train_data_counts)

#################

#TF-IDF
tfidf = TfidfTransformer()
#Обучение на тренировочных данных
d_train_tfidf = tfidf.fit_transform(lem_train_data_counts)
#Преобразование текста в Tf-Idf матрицу на тестовых данных
d_test_tfidf = tfidf.transform(lem_test_data_counts)

print(d_test_tfidf)

#################

#Обучаем модели логистической регрессии и случайного леса на обучающей выборке, применяем их к тестовым данным
label_maps = {"positive":1, "negative":0}

#Сопоставление меток с числами
D_train_encoded = lem_train_data['label'].map(label_maps)
D_test_encoded = lem_test_data['label'].map(label_maps)

#Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(d_train_tfidf, D_train_encoded)

#Случайный лес
rand_forest = RandomForestClassifier()
rand_forest.fit(d_train_tfidf, D_train_encoded)

#Предсказания
predict_lin_reg_train = log_reg.predict(d_train_tfidf)
predict_rand_for_train = rand_forest.predict(d_train_tfidf)

predict_lin_reg_test = log_reg.predict(d_test_tfidf)
predict_rand_for_test = rand_forest.predict(d_test_tfidf)

from sklearn.metrics import classification_report
#выводим полученные результаты по показателям
print("Обучающие данные\n")
print("Линейная регрессия\n", classification_report(D_train_encoded,predict_lin_reg_train))
print("Случайный лес\n", classification_report(D_train_encoded,predict_rand_for_train))

print("Тестовые данные\n")
print("Линейная регрессия\n", classification_report(D_test_encoded,predict_lin_reg_test))
print("Случайный лес\n", classification_report(D_test_encoded,predict_rand_for_test))

#######################

#Выявляем наиболее важные признаки(слова)
n = 15

log_reg_coef = log_reg.coef_[0]
log_reg_ind = abs(log_reg_coef).argsort()[::-1]
log_reg_names = vectorizer.get_feature_names_out()
log_reg_top_names = [log_reg_names[index] for index in log_reg_ind[:n]]
print("Топ 25 наиболее значимых слов для логистической регрессии:\n", log_reg_top_names)

rand_forest_coef = rand_forest.feature_importances_
rand_forest_ind = abs(rand_forest_coef).argsort()[::-1]
rand_forest_names = vectorizer.get_feature_names_out()
rand_forest_top_names = [rand_forest_names[index] for index in rand_forest_ind[:n]]
print("Топ 25 наиболее значимых слов для случайного леса:\n", rand_forest_top_names)